In [ ]:
!pip install --upgrade torch torchvision
!pip install facenet-pytorch

In [ ]:
import os
import cv2
import torch
from facenet_pytorch import MTCNN
from PIL import Image
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import json
# 메타데이터 로드
metadata_file = '/Users/gible/dataset/Video_maindata/metadata.json' # 파일 경로 조정
with open(metadata_file, 'r') as f:
    metadata = json.load(f)


In [ ]:
# 출력 폴더 생성
os.makedirs(output_folder, exist_ok=True)

In [ ]:
# 비디오 처리 함수 정의
def process_video(video_path, output_folder, mtcnn, metadata, frame_interval=10, resize_dim=(224, 224)):
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    extracted_count = 0
    video_name = os.path.splitext(os.path.basename(video_path))[0]

    # 비디오별 폴더 생성
    video_output_folder = os.path.join(output_folder, video_name)
    os.makedirs(video_output_folder, exist_ok=True)

    # 메타데이터에서 라벨 가져오기
    label = metadata.get(f"{video_name}.mp4", {}).get("label", "REAL")
    prefix = "FAKE_" if label == "FAKE" else ""

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # 프레임 간격에 따라 샘플링
        if frame_count % frame_interval == 0:
            # OpenCV BGR -> PIL RGB 변환
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(frame_rgb)

            # 얼굴 감지
            boxes, _ = mtcnn.detect(pil_img)
            if boxes is not None:
                for i, box in enumerate(boxes):
                    face = pil_img.crop((box[0], box[1], box[2], box[3])).resize(resize_dim)
                    face_filename = f"{prefix}{video_name}_face_{extracted_count}.jpg"
                    face.save(os.path.join(video_output_folder, face_filename))
                    extracted_count += 1

        frame_count += 1

    cap.release()

In [ ]:
import cv2
import numpy as np

def pad_to_divisible(frame, divisor=32):
    h, w, _ = frame.shape
    new_h = (h // divisor + 1) * divisor if h % divisor != 0 else h
    new_w = (w // divisor + 1) * divisor if w % divisor != 0 else w
    padded_frame = np.zeros((new_h, new_w, 3), dtype=frame.dtype)
    padded_frame[:h, :w, :] = frame
    return padded_frame

In [ ]:
video_files = []
for root, _, files in os.walk(video_folder):
    for file in files:
        if file.endswith('.mp4'):
            video_files.append(os.path.join(root, file))

if not video_files:
    print("지정된 폴더에서 .mp4 파일을 찾을 수 없습니다.")
else:
    print(f"총 {len(video_files)}개의 비디오 파일이 발견되었습니다.")

# 모든 비디오 파일 처리
for video_file in tqdm(video_files, desc="비디오 전처리 중"):
    try:
        process_video(video_file, output_folder, mtcnn, metadata)
        print(f"{os.path.basename(video_file)} 처리 완료")
    except Exception as e:
        print(f"{os.path.basename(video_file)} 처리 오류: {e}")

print("비디오 전처리가 완료되었습니다.")

### 증강: 컬러조정, 가우시안블러. 픽셀드롭아웃, 어파인변환, 페르스펙티브 변환

In [ ]:
import os
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
from torchvision import transforms

# 경로 설정
input_folder = '/Users/gible/dataset/Image_maindata'
output_folder = '/Users/gible/dataset/AugmentedData'

# 증강 함수 정의
def augment_image(image, method):
    if method == "color_jitter":
        transform = transforms.ColorJitter(brightness=0.5, contrast=0.5, saturation=0.5, hue=0.1)
        return transform(image)
    elif method == "gaussian_blur":
        return image.filter(ImageFilter.GaussianBlur(radius=2))
    elif method == "random_erasing":
        transform = transforms.RandomErasing(p=1.0)
        image_tensor = transforms.ToTensor()(image).unsqueeze(0)
        erased_image = transform(image_tensor).squeeze(0)
        return transforms.ToPILImage()(erased_image)
    elif method == "affine":
        transform = transforms.RandomAffine(degrees=15, translate=(0.1, 0.1), scale=(0.9, 1.1))
        return transform(image)
    elif method == "perspective":
        transform = transforms.RandomPerspective(distortion_scale=0.5, p=1.0)
        return transform(image)
    elif method == "flip":
        return ImageOps.mirror(image)
    else:
        return image

# 폴더 순회 및 증강
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for root, _, files in os.walk(input_folder):
    for file in files:
        if file.lower().endswith('.jpg') and not file.startswith('FAKE'):
            file_path = os.path.join(root, file)
            image = Image.open(file_path).convert("RGB")

            # 증강 기법 리스트
            methods = ["color_jitter", "gaussian_blur", "random_erasing", "affine", "perspective", "flip"]

            for method in methods:
                augmented_image = augment_image(image, method)
                augmented_file_name = f"{os.path.splitext(file)[0]}_{method}.jpg"
                output_path = os.path.join(output_folder, augmented_file_name)

                augmented_image.save(output_path)
                print(f"{augmented_file_name} 생성 완료")

In [ ]:
import os

# 파일 경로 설정
image_folder = '/content/drive/Othercomputers/내 MacBook Pro/Image_maindata'
# 파일 비율 계산
total_files = 0
fake_files = 0

# 폴더 순회
for root, _, files in os.walk(image_folder):
    for file in files:
        if file.lower().endswith('.jpg'):
            total_files += 1
            if file.startswith('FAKE'):
                fake_files += 1

# 비율 계산
fake_ratio = (fake_files / total_files) * 100 if total_files > 0 else 0
real_ratio = 100 - fake_ratio if total_files > 0 else 0

print(f"총 이미지 파일 수: {total_files}")
print(f"FAKE로 시작하는 파일 수: {fake_files}")
print(f"FAKE 파일 비율: {fake_ratio:.2f}%")
print(f"REAL 파일 비율: {real_ratio:.2f}%")

총 이미지 파일 수: 146149
FAKE로 시작하는 파일 수: 106957
FAKE 파일 비율: 73.18%
REAL 파일 비율: 26.82%


In [ ]:
import os
import random
import uuid
from PIL import Image
import torch
import torchvision.transforms as transforms

# 입력 및 출력 폴더 경로 설정
input_folder = "/Users/gible/dataset/Image_maindata"
exclude_folder = "/Users/gible/dataset/Image_maindata/AugmentedData"
output_folder = "/Users/gible/dataset/augmented3"

# 증강 함수 정의
def augment_image(image, file_name, output_folder):
    augmentations = {
        "horizontal_flip": transforms.RandomHorizontalFlip(p=1.0),
        "brightness": transforms.ColorJitter(brightness=0.5),
        "rotation": transforms.RandomRotation(degrees=30),
        "resize_crop": transforms.RandomResizedCrop(size=(224, 224)),
        "noise": lambda img: Image.fromarray(
            (torch.clamp(
                transforms.ToTensor()(img) + torch.randn_like(transforms.ToTensor()(img)) * 0.05, 0, 1
            ).mul(255).byte().numpy().transpose(1, 2, 0))
        ),
        "grayscale": transforms.RandomGrayscale(p=1.0),
    }

    # 증강 수행
    for name, transform in augmentations.items():
        try:
            augmented = transform(image)
            unique_name = f"{name}_{uuid.uuid4().hex}_{file_name}"
            augmented.save(os.path.join(output_folder, unique_name))
        except Exception as e:
            print(f"{file_name}의 {name} 증강 오류: {e}")

# 폴더 생성
os.makedirs(output_folder, exist_ok=True)

# 대상 파일 필터링
all_files = [
    os.path.join(root, file)
    for root, _, files in os.walk(input_folder)
    for file in files
    if file.lower().endswith(".jpg")
    and not file.startswith("FAKE")
    and not root.startswith(exclude_folder)
]

# 50% 무작위 선택
selected_files = random.sample(all_files, len(all_files) // 2)

# 증강 적용
for image_path in selected_files:
    file = os.path.basename(image_path)
    try:
        with Image.open(image_path) as img:
            # RGB 변환 필수
            img = img.convert("RGB")
            augment_image(img, file, output_folder)
            print(f"{file} 처리 완료")
    except Exception as e:
        print(f"{file} 처리 오류: {e}")

print("이미지 증강 완료.")

### 학습

In [ ]:
!pip install efficientnet-pytorch pretrainedmodels

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16424 sha256=edcb12ef1b9ef835ea33f4a2b081e1f2cf5a5f959a52921281013d04d73ad44e
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60944 sha256=9159179394e362c28e219486dfe1b605da9d76e9f43dc41a01d80e230735058e
  Stored in directory: /root/.cache/pip/wheels/35/cb/a5/8f534c60142835bfc889f9a482e4a67e0b817032d9c6883b64
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
import os
import ssl
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, random_split
from torchvision import transforms, models
from torchvision.models import ResNet50_Weights
from efficientnet_pytorch import EfficientNet
from pretrainedmodels import xception
from PIL import Image
from tqdm import tqdm

# SSL 인증서 검증 비활성화
ssl._create_default_https_context = ssl._create_unverified_context

# CUDA 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.backends.cudnn.benchmark = True

print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU 이름: {torch.cuda.get_device_name(0)}")

# 사용자 정의 데이터셋 클래스
class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for root, _, files in os.walk(root_dir):
            for file in files:
                if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                    file_path = os.path.join(root, file)
                    self.image_paths.append(file_path)
                    self.labels.append(0 if file.startswith("FAKE") else 1)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        label = self.labels[idx]
        try:
            image = Image.open(image_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, label
        except Exception as e:
            print(f"Error loading image {image_path}: {e}")
            return None, None

# 데이터 전처리 정의
transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

# 데이터셋 로드
image_folder = '/content/drive/Othercomputers/내 MacBook Pro/Image_maindata'
dataset = CustomDataset(root_dir=image_folder, transform=transform)

# 데이터셋 분할
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 데이터 로더 생성
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=8, pin_memory=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=8, pin_memory=True, persistent_workers=True)

# 모델 초기화
xception_model = xception(num_classes=1000, pretrained='imagenet')
xception_model.last_linear = nn.Linear(xception_model.last_linear.in_features, 2)
xception_model = nn.DataParallel(xception_model.to(device))

efficientnet_model = EfficientNet.from_pretrained('efficientnet-b0', num_classes=2)
efficientnet_model = nn.DataParallel(efficientnet_model.to(device))

resnet_model = models.resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)
resnet_model.fc = nn.Linear(resnet_model.fc.in_features, 2)
resnet_model = nn.DataParallel(resnet_model.to(device))

# 손실 및 최적화 함수
criterion = nn.CrossEntropyLoss()
optimizer = Adam(
    list(xception_model.parameters()) +
    list(efficientnet_model.parameters()) +
    list(resnet_model.parameters()),
    lr=3e-4
)

# 학습률 스케줄러 정의
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=2, verbose=True)

# 학습 루프 정의
scaler = torch.cuda.amp.GradScaler()

def train_ensemble_model(models, criterion, optimizer, scheduler, num_epochs=7):
    for epoch in range(num_epochs):
        train_loss, train_correct = 0.0, 0

        for model in models:
            model.train()

        for images, labels in tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs}"):
            images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)

            with torch.cuda.amp.autocast():
                outputs = torch.stack([model(images) for model in models]).mean(dim=0)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_correct += (preds == labels).sum().item()

        train_acc = train_correct / len(train_loader.dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {train_loss/len(train_loader):.4f}, Accuracy: {train_acc:.4f}")

        val_loss, val_correct = 0.0, 0

        for model in models:
            model.eval()

        with torch.no_grad():
            for images, labels in tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}"):
                images, labels = images.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                outputs = torch.stack([model(images) for model in models]).mean(dim=0)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()

        val_acc = val_correct / len(val_loader.dataset)
        print(f"Validation Loss: {val_loss/len(val_loader):.4f}, Validation Accuracy: {val_acc:.4f}")

        scheduler.step(val_loss)

# 학습 시작
models = [xception_model, efficientnet_model, resnet_model]
train_ensemble_model(models, criterion, optimizer, scheduler, num_epochs=7)

# 모델 저장
output_dir = '/content/drive/Othercomputers/내 MacBook Pro/Image_maindata'
os.makedirs(output_dir, exist_ok=True)

torch.save(xception_model.state_dict(), os.path.join(output_dir, 'xception_model.pth'))
torch.save(efficientnet_model.state_dict(), os.path.join(output_dir, 'efficientnet_model.pth'))
torch.save(resnet_model.state_dict(), os.path.join(output_dir, 'resnet_model.pth'))

print("모델이 성공적으로 저장되었습니다.")

Using device: cuda
GPU 이름: NVIDIA L4


Downloading: "http://data.lip6.fr/cadene/pretrainedmodels/xception-43020ad28.pth" to /root/.cache/torch/hub/checkpoints/xception-43020ad28.pth
100%|██████████| 87.4M/87.4M [04:17<00:00, 356kB/s]
Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b0-355c32eb.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b0-355c32eb.pth
100%|██████████| 20.4M/20.4M [00:00<00:00, 39.7MB/s]


Loaded pretrained weights for efficientnet-b0


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 214MB/s]
/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(
<ipython-input-3-fc14d2801085>:101: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Training Epoch 1/7:   0%|          | 0/914 [00:00<?, ?it/s]<ipython-input-3-fc14d2801085>:113: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Training Epoch 1/7: 100%|██████████| 914/914 [2:44:41<00:00, 10.81s/it]


Epoch [1/7], Loss: 0.1365, Accuracy: 0.9477


...

Validation Loss: 0.0782, Validation Accuracy: 0.9758


Training Epoch 7/7: 100%|██████████| 914/914 [03:30<00:00,  4.35it/s]


Epoch [7/7], Loss: 0.0327, Accuracy: 0.9889


Validation Epoch 7/7: 100%|██████████| 229/229 [00:50<00:00,  4.50it/s]


Validation Loss: 0.0819, Validation Accuracy: 0.9726
모델이 성공적으로 저장되었습니다.
